# Analysis of the National Survey of Health

## Jason Piccone, Ph.D.

## Part I: Data Preparation

# Import Libraries

In [ ]:
from collections import Counter
from pandas import DataFrame
from matplotlib import style 
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random


# Parameters

In [ ]:

path = "/home/paco/Documents/data science/DS projects/national survey on drug use and health 2012/ICPSR_34933/DS0001/"
var_threshold = 0.05  # will remove features with less than 5% variance 
f_name = 'Data.tsv'

# Read Data

In [ ]:

df =  pd.read_csv(path+f_name, sep="\t")

print(df.shape)


# Basic feature reduction/management

In [ ]:
# 1. focus on just the adult population
df = df.loc[df['CATAG6'] != 1]

print(df.shape)

    
# 2. Only some features are relevant for the adult population:
adult_features = ['RSKPKCIG', 'RSKMJOCC', 'RKTRYLSD', 'RKTRYHER', 
'RKCOCOCC', 'RK5ALWK', 'RSKDIFMJ', 'RKDIFLSD', 'RKDIFCOC', 'RKDIFCRK', 'RKFQDNGR', 'RKFQRSKY', 'RKFQPBLT', 'RKFQDBLT', 'NMERTMT2', 'SNYSELL', 'SNYSTOLE', 'SNYATTAK',
'SNFAMJEV', 'SNRLGSVC', 'SNRLGIMP', 'SNRLDCSN', 'SNRLFRND', 'DSTNRV30', 'DSTHOP30', 'DSTRST30',
'DSTCHR30', 'DSTEFF30', 'DSTNGD30', 'IRHHSIZ2', 'IRKI17_2', 'IRHH65_2', 'IRFAMSZ2', 'IRKIDFA2',
'IRPINC3', 'IRFAMIN3', 'AGE2', 'HEALTH', 'IREDUC2', 'CIGEVER', 'SNFEVER', 'CIGAREVR', 
'ALCEVER', 'MJEVER', 'COCEVER', 'CRKEVER', 'PCP', 'PEYOTE', 'MESC', 'PSILCY', 'ECSTASY',
'HALNOLST', 'AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER', 'SOLVENT', 'LGAS', 'NITOXID', 'SPPAINT',
'AEROS', 'INHNOLST', 'DARVTYLC', 'PERCTYLX', 'ANLNOLST', 'KLONOPIN', 'XNAXATVN', 'VALMDIAZ', 
'TRNEVER', 'METHDES', 'DIETPILS', 'RITMPHEN', 'STMNOLST', 'STMEVER', 'SEDEVER', 'ADDERALL',
'AMBIEN', 'COLDMEDS', 'KETAMINE', 'RSKSELL', 'BOOKED', 'PROBATON', 'TXEVER', 'INHOSPYR',
'AUINPYR', 'AUOPTYR', 'AURXYR', 'AUUNMTYR', 'SUICTHNK', 'ADDPREV', 'IRFAMSOC', 
'MEDICARE', 'PRVHLTIN', 'HLCNOTYR', 'SERVICE', 'IRSEX', 'SCHENRL', 
'IRMARIT', 'NEWRACE2', 'EMPSTATY','CATAG6',
'COUTYP2']


df = df[adult_features]
print(df.shape)

In [ ]:
pd.set_option('display.max_columns', 10) 

# do a quick quality check of the remaining features
for feature in df.iloc[:, :2]:   # selecting just a subset for presentational purposes


    print('**************************************\n')
    print(feature + ' unique values = ' + str(df[feature].unique()))           
    print(feature + ' value counts = ' + str(df[feature].value_counts())+'\n')


# Significant recoding is necessary:

In [ ]:
# function to recode multiple variables

def restructure_features(df):
    #(a) Convert the following variables from 3 (logically assigned "yes") to 1 ("yes)
    vars1 = ['PCP', 'PEYOTE', 'PSILCY', 'ECSTASY', 'AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER', 'NITOXID',
             'AEROS', 'DARVTYLC', 'PERCTYLX', 'KLONOPIN', 
             'METHDES', 'DIETPILS', 'RITMPHEN',  'BOOKED']
             
    for col in vars1:
        df.loc[df[col] == 3, col].copy = 1

    #(c) Convert 4 ("No, logically assigned") =2 ("no")
    vars3 = ['ANLNOLST', 'STMNOLST'] 
    for col in vars3:
        df.loc[df[col] == 4, col].copy = 2


    #(d) Convert 81 ("never used X type of drug, logically assigned" and 91 ("never used 
    # X type of drug") = 2 ("no")
    vars4 = ['TRNEVER', 'STMEVER', 'SEDEVER'] 
    for col in vars4:
        df.loc[df[col] == 81, col].copy = 2
    for col in vars4:
        df.loc[df[col] == 91, col].copy = 2
   
       
    #(h) Convert SCHENRL 3, 5, 11 ("Yes, logically assigned") = yes(1)
    df['SCHENRL'][df.SCHENRL == 3].copy = 1
    df['SCHENRL'][df.SCHENRL == 5].copy = 1
    df['SCHENRL'][df.SCHENRL == 11].copy = 1

    
    #(i) Convert 91 ("Ever used") -> 2 ("no") 
    vars7 = ['CRKEVER', 'PCP', 'PEYOTE', 'MESC', 'PSILCY', 'ECSTASY', 'HALNOLST', 'AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER',
      'SOLVENT', 'LGAS', 'NITOXID', 'SPPAINT', 'AEROS', 'INHNOLST', 'PERCTYLX', 'DARVTYLC',
      'ANLNOLST', 'KLONOPIN', 'XNAXATVN', 'VALMDIAZ', 'TRNEVER', 'METHDES', 'DIETPILS', 
      'RITMPHEN', 'STMNOLST', 'STMEVER', 'SEDEVER']  
    for col in vars7:
        df.loc[df[col] == 91, col].copy = 2  
    
      
    #onvert 81 ("Never used X type of drug")-> 2("No")
    vars8 = ['PERCTYLX', 'DARVTYLC', 'ANLNOLST', 'KLONOPIN', 'XNAXATVN',
        'VALMDIAZ', 'TRNEVER', 'METHDES', 'DIETPILS', 'RITMPHEN', 'STMNOLST', 'STMEVER', 'SEDEVER']      
    for col in vars8:
        df.loc[df[col] == 81, col].copy = 2

    #(j) Convert 94, 97 and 99 to Nan
    #df = df.replace(94, np.nan)
    #df = df.replace(97, np.nan)
    #df = df.replace(99, np.nan)
    df.where(df < 80, np.nan, inplace=True)


    return(df)

# Apply feature cleaning:
df_cleaned = restructure_features(df)
print(df_cleaned.shape)

# Data visualization examples

In [ ]:
style.use('ggplot')

vars = ['CIGEVER','COCEVER']   #just doing two for demonstration purposes

for var in vars:
    
    if var == 'CIGEVER':
        title = 'Smoked a Cigarette'
    else:
        title = 'Used Cocaine'
    
    df_cleaned3 = df_cleaned[var]
    df_cleaned3.dropna(inplace=True)
    fig = plt.figure()    
    counts = (Counter(df_cleaned3)).values()
    ax1 = fig.add_subplot(111)
    objects = ('Yes','No')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, counts, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('Have You Ever {}'.format(title))

    plt.show()



# Standardize the data

In [ ]:

# Standardize quantitative variables, and dummy code qualitative (where necessary). 

def standardize_features(df):
    
    #split dataframe into quantitative data and categorical data
    
    #1 standardize quantitative variables:
    quant_vars = [  
    'RSKPKCIG', 'RSKMJOCC', 'RKTRYLSD', 
    'RKTRYHER', 'RKCOCOCC', 'RK5ALWK', 'RSKDIFMJ', 'RKDIFLSD', 'RKDIFCOC', 'RKDIFCRK',  
    'RKFQDNGR', 'RKFQRSKY', 'RKFQPBLT', 'RKFQDBLT', 'NMERTMT2', 'SNYSELL', 'SNYSTOLE', 
    'SNYATTAK', 'SNFAMJEV', 'SNRLGSVC', 'SNRLGIMP', 'SNRLDCSN', 'SNRLFRND', 'DSTNRV30', 
    'DSTHOP30','DSTRST30', 'DSTCHR30', 'DSTEFF30','DSTNGD30', 'IRHHSIZ2', 'IRKI17_2', 
    'IRHH65_2', 'IRFAMSZ2', 'IRKIDFA2', 'IRPINC3', 'IRFAMIN3', 'AGE2', 'HEALTH', 'IREDUC2']

    # assemble the quantitative variables, and drop na
    dfQuant = df[quant_vars]
    dfQuant.dropna(inplace=True)
    dfQuant = preprocessing.scale(dfQuant)
    dfQuant = pd.DataFrame(data=dfQuant, columns=quant_vars)
    

    #2 recode binary categorical variables:
    cat_features = ['CIGEVER','SNFEVER','CIGAREVR','ALCEVER','MJEVER','COCEVER','CRKEVER',
    'PCP','PEYOTE','MESC','PSILCY','ECSTASY','HALNOLST','AMYLNIT','CLEFLU', 'GAS','GLUE',
    'ETHER','SOLVENT','LGAS','NITOXID','SPPAINT','AEROS','INHNOLST','DARVTYLC', 'PERCTYLX', 
    'ANLNOLST', 'KLONOPIN', 'XNAXATVN', 'VALMDIAZ', 'TRNEVER', 'METHDES', 'DIETPILS', 'RITMPHEN', 
    'STMNOLST', 'STMEVER', 'SEDEVER','ADDERALL', 'AMBIEN', 'COLDMEDS', 'KETAMINE', 'RSKSELL', 
    'BOOKED', 'PROBATON','TXEVER', 'INHOSPYR', 'AUINPYR','AUOPTYR', 'AURXYR', 'AUUNMTYR', 
    'SUICTHNK', 'ADDPREV', 'IRFAMSOC', 'MEDICARE', 'PRVHLTIN', 'HLCNOTYR',  'SERVICE', 'IRSEX', 
    'SCHENRL']
    
    # assemble the categorical variables
    dfCat = df[cat_features]
    dfCat = dfCat.replace(1,0)
    dfCat = dfCat.replace(2,1)
    
    
    #3 dummy code non-binary categorical variables 
 
    MARITAL = pd.get_dummies(df['IRMARIT'])
    MARITAL.columns = ['MARRIED','WIDOWED','DIVORCED','NEVER_MARRIED']

    RACE = pd.get_dummies(df['NEWRACE2'])
    RACE.columns = ['WHITE','BLACK','NATIVEAM','PACISL','ASIAN','MULTIPLE','HISPANIC']

    EMPLOY = pd.get_dummies(df['EMPSTATY'])
    EMPLOY.columns =['FULLTIME','PARTTIME','UNEMPLOYED','OTHER'] #,'EMPLOYMENT_SKIPPED']

    AGECAT = pd.get_dummies(df['CATAG6'])
    AGECAT.columns =['AGE18_25','AGE26_34','AGE35_49','AGE50_64','AGE65']

    COUNTY = pd.get_dummies(df['COUTYP2'])
    COUNTY.columns = ['COUNTY_LARGE','COUNTY_SMALL','COUNTY_NONMETRO']
    
    # 4. Combine them back together
    total_df = pd.concat([dfQuant, dfCat, MARITAL,RACE,EMPLOY,AGECAT,COUNTY], axis=1)
    
    # 5 Remove NA
    total_df.dropna(inplace=True)
    
    # 6. Remove features with tiny variances
    features_prior = total_df.shape[1]
    def variance_threshold_selector(df, threshold=var_threshold):
        selector = VarianceThreshold(threshold)
        selector.fit(df)
        
        return(df[df.columns[selector.get_support(indices=True)]])

    total_df = variance_threshold_selector(total_df,var_threshold)
    
    # print the change in features due to low variance
    features_post = total_df.shape[1]
    features_changed = features_prior - features_post
    print('Features eliminated for lacking variance = {0} out of {1}'.format(features_changed,features_prior))

        
    return (total_df)


combined_df = standardize_features(df_cleaned)
print(combined_df.shape)


In [ ]:
# save to file for use in part II
combined_df.to_csv(path+'combined_df.csv', index=False)  
